<a href="https://colab.research.google.com/github/zatyhan/spam-text-classification/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  # %tensorflow_version 2.x
  # !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


# print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data= pd.read_csv(train_file_path, delimiter='\t', names= ['class', 'text'])

In [ ]:
train_labels= train_data['class']
train_data= train_data.drop(columns='class')

In [ ]:
train_labels= np.array([0 if k =="ham" else 1 for k in train_labels], dtype=np.float32)
train_labels

In [ ]:
train_data

In [ ]:
vectorizer= TfidfVectorizer(max_features= 700)
train_transformed= vectorizer.fit_transform(train_data['text']).toarray()

In [ ]:
model = tf.keras.Sequential(
    [
        keras.layers.Flatten(input_shape=(train_transformed.shape[1],)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ]
)
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(train_transformed, train_labels, batch_size=16, epochs= 100)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred=model.predict(vectorizer.transform([pred_text]).toarray())

  if round(pred[0][0])==1:
    prediction="spam"
  else:
    prediction="ham"

  return (pred_text, prediction)

In [ ]:
predict_message("how are you doing today")

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
